In [5]:
####IMPORTAMOS LIBRERÍAS
import pandas as pd
import os
import numpy as np
import requests
import geopandas as gpd 
from shapely.geometry import Point

In [22]:
#### DEFINIMOS FUNCIONES 
def distance_m(mercator_start, mercator_finish):
    return mercator_start.distance(mercator_finish)


def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

#FUNCIÓN QUE RECIBE DATOS DE BICIMAD Y DEVUELVE EL DATASET CON EL MODELO DE DATOS QUE BUSCAMOS
#FUNCIÓN QUE EXPORTA DATOS DE BICIMAD Y DEVUELVE EL DATASET CON EL MODELO DE DATOS QUE BUSCAMOS


def getdata_bicimad(json):
    bicimad_data = pd.read_json(json)
    bicimad_data = bicimad_data.drop(columns=['id','light',"number","activate","no_available","total_bases", "dock_bikes","free_bases","reservations_count","geometry_type"])
    bicimad_data['LONGITUD']=bicimad_data['geometry_coordinates'].map(lambda x:x.split(',')[0].replace("[","")).astype(float)
    bicimad_data['LATITUD']=bicimad_data['geometry_coordinates'].map(lambda x:x.split(',')[1].replace("]","")).astype(float)
    bicimad_data = bicimad_data.drop(columns=['geometry_coordinates'])
    bicimad_data = bicimad_data.rename(columns={'LATITUD':'lat_finish','LONGITUD':'long_finish','name':'BiciMAD station','address':'Station location'})
    bicimad_data["PUNTO B"] = bicimad_data.apply(lambda x: to_mercator(x["lat_finish"],x["long_finish"]), axis = 1)
    result = bicimad_data

    return result


#FUNCIÓN QUE RECIBE URL DE AYUNTAMIENTO Y DEVUELVE EL DATASET DE COLEGIOS CON EL MODELO DE DATOS LIMPIO

def getdata_public_school(url):
    colegios_publicos_data = requests.get(url)
    colegios_publicos_data = colegios_publicos_data.json()
    colegios_publicos_data = pd.json_normalize(colegios_publicos_data['@graph'])
    colegios_publicos_data = colegios_publicos_data.drop(columns=["@id","@type","id","relation","address.district.@id","address.area.@id","address.locality","address.postal-code","organization.organization-desc","organization.accesibility","organization.schedule","organization.services","organization.organization-name"])
    result = colegios_publicos_data.assign(Tipo_Centro = "Colegios Publicos")
    return result


#FUNCIÓN QUE RECIBE URL DE AYUNTAMIENTO Y DEVUELVE EL DATASET DE ESCUELAS CON EL MODELO DE DATOS LIMPIO

def getdata_escuelas_infantiles(url):
    escuelas_infantiles_data = requests.get(url)
    escuelas_infantiles_data = escuelas_infantiles_data.json()
    escuelas_infantiles_data = pd.json_normalize(escuelas_infantiles_data['@graph'])
    escuelas_infantiles_data = escuelas_infantiles_data.drop(columns=["@id","@type","id","relation","address.district.@id","address.area.@id","address.locality","address.postal-code","organization.organization-desc","organization.accesibility","organization.schedule","organization.services","organization.organization-name"])
    result =escuelas_infantiles_data.assign(Tipo_Centro = "Escuelas Infantiles")
    return result

#FUNCIÓN QUE MERGEA DATASET DE AYUNTAMIENTO Y LOS LIMPIA Y AÑADE COLUMNA PUNTO A

def concat_dataset_ayuntamiento(data1,data2):
    ayuntamiento_data = pd.concat([data1, data2], axis=0)
    ayuntamiento_data['address.street-address']=ayuntamiento_data['address.street-address'].str.title()
    ayuntamiento_data = ayuntamiento_data.rename(columns={'location.latitude':'lat_start','location.longitude':'long_start','title':'Place of interest','address.street-address':'Place address'})
    ayuntamiento_data["PUNTO A"] = ayuntamiento_data.apply(lambda x: to_mercator(x["lat_start"],x["long_start"]), axis = 1)
    result = ayuntamiento_data
    return result

#FUNCIÓN QUE MERGEA DATASET DEL PROYECTO Y CREA LA COLUMNA CON EL RESULTADO DE LA DISTANCIA EN METROS ENTRE LOS PUNTOS A Y B

def concat_dataset_proyect(data1,data2):
    proyect_data = pd.merge(data1, data2 ,how="cross")
    proyect_data['Distancia'] = proyect_data.apply(lambda x: distance_m(x["PUNTO A"], x["PUNTO B"]), axis = 1)
    result = proyect_data.drop(['lat_start','long_start', 'long_finish', 'lat_finish','PUNTO A','PUNTO B'], axis='columns')
    # proyect_data['address.street-address']=ayuntamiento_data['address.street-address'].str.title() ##ELIMINAR SI EJECUTA ##
    return result


## FUNCIÓN PARA RECIBIR EL INPUT Y DEVOLVER LA DISTANCIA MÍNIMA

def result_one(data):
    all_results = data[data["Place of interest"] == input('Pon el lugar de interés: ')]
    selection = all_results[all_results['Distancia'] == all_results['Distancia'].min()] # Comprobar si es Distancia o Distance
    result = selection[["Place of interest","Tipo_Centro","Place address","BiciMAD station","Station location"]]
    return result

## FUNCIÓN PARA RECIBIR EL INPUT Y DEVOLVER TODOS LOS RESULTAOS

def result_all(data):
    all_results = data[data["Place of interest"] == input('Pon el lugar de interés: ')]
    selection = all_results[all_results['Distancia'] == all_results['Distancia']]
    result = selection[["Place of interest","Tipo_Centro","Place address","BiciMAD station","Station location"]]
    return result

In [14]:
#### DEFINIMOS CONSTANTES
# Estas constantes pueden ser argumentos para argparse?

json_bicimad = "../data/bicimad.json"
url_colegios_json ='https://datos.madrid.es/egob/catalogo/202311-0-colegios-publicos.json'
url_escuelas_json ='https://datos.madrid.es/egob/catalogo/202318-0-escuelas-infantiles.json'


In [15]:
### PROGRAMA A EJECUTAR:

# Punto de partida: obtenemos los 3 dataset
data_bicimad = getdata_bicimad(json_bicimad)
data_kindergarten = getdata_escuelas_infantiles(url_escuelas_json)
data_public_school =getdata_public_school(url_colegios_json)


In [16]:
# 1º Merge del Dataset ayuntamientos
data_city_hall = concat_dataset_ayuntamiento(data_kindergarten,data_public_school)

In [17]:
# 2º Merge dataset de ayuntamiento y bicimad para la consulta
data_project = concat_dataset_proyect(data_city_hall,data_bicimad)
data_project

,Place of interest,Place address,Tipo_Centro,BiciMAD station,Station location,Distancia
0,Escuela infantil municipal Agua Dulce,Calle Leñeros 25,Escuelas Infantiles,Puerta del Sol A,Puerta del Sol nº 1,4187.097788
1,Escuela infantil municipal Agua Dulce,Calle Leñeros 25,Escuelas Infantiles,Puerta del Sol B,Puerta del Sol nº 1,4181.568956
2,Escuela infantil municipal Agua Dulce,Calle Leñeros 25,Escuelas Infantiles,Miguel Moya,Calle Miguel Moya nº 1,3746.865547
3,Escuela infantil municipal Agua Dulce,Calle Leñeros 25,Escuelas Infantiles,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,2659.877980
4,Escuela infantil municipal Agua Dulce,Calle Leñeros 25,Escuelas Infantiles,Malasaña,Calle Manuela Malasaña nº 5,2938.700507
...,...,...,...,...,...,...
81571,Colegio Público Álvaro de Bazan,Calle Zurich 10,Colegios Publicos,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,13584.465918
81572,Colegio Público Álvaro de Bazan,Calle Zurich 10,Colegios Publicos,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,13571.244295
81573,Colegio Público Álvaro de Bazan,Calle Zurich 10,Colegios Publicos,Facultad Biología,Calle José Antonio Novais frente al nº 12,13719.637536
81574,Colegio Público Álvaro de Bazan,Calle Zurich 10,Colegios Publicos,Facultad Derecho,Avenida Complutense nº 23,14014.582919


In [18]:
# 3º Ejecutamos Argparse para que nos devuelva: a) la estación más cercana b): todas las estaciones de menor a mayor
result_one(data_project)
#Escuela infantil municipal Doña Francisquita

Pon el lugar de interés: Escuela infantil municipal Doña Francisquita


,Place of interest,Tipo_Centro,Place address,BiciMAD station,Station location,Distancia
1816,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Carlota O’Neill,Calle Carlota O'Neill nº 32,4418.494579


In [23]:
result_all(data_project)

Pon el lugar de interés: Escuela infantil municipal Doña Francisquita


,Place of interest,Tipo_Centro,Place address,BiciMAD station,Station location
1584,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Puerta del Sol A,Puerta del Sol nº 1
1585,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Puerta del Sol B,Puerta del Sol nº 1
1586,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Miguel Moya,Calle Miguel Moya nº 1
1587,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3
1588,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Malasaña,Calle Manuela Malasaña nº 5
...,...,...,...,...,...
1843,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...
1844,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...
1845,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Facultad Biología,Calle José Antonio Novais frente al nº 12
1846,Escuela infantil municipal Doña Francisquita,Escuelas Infantiles,Calle Cambados 1,Facultad Derecho,Avenida Complutense nº 23
